<a href="https://colab.research.google.com/github/hepuliu/Masters_Thesis/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Master Thesis Simulation - CNN Model

*Flood Prevention with Machine Learining - Hepu Liu*

---

**This Notebook is Dedicated to the CNN Model Simulations**

### Overall Project Simulation Steps
1. Process discharge data from Waldangelbach Station

2. Process precipitation data from Baiertal  Station

3. Build Prediction Models

4. Evaluation of NSE

### Variable Naming Conventions

- Weather Stations Naming: ('p' for precipitation, 'd' for discharge, 'a' for different stations, 'r' for result)

  - da: Waldangelbach Station
  - pa: Baiertal Station
  - pr: combined/resulting precipitation
  - dr: predicted/resulting discharge

- Variable Naming Coventions: 
  - df: data frame
  - trs: training set
  - tes: testing set
  - lstm: LSTM
  - cnn: CNN
  - lstmss: LSTM-seq2sqe


## Importing Libraries

In [15]:
# importing libraries
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame
from pandas import concat
from math import sqrt
from numpy import concatenate
from numpy import loadtxt
from numpy import array
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount

<function google.colab.drive.mount>

## Importing Datasets

In [16]:
# import datafram for CNN
df_cnn = pd.read_csv('/content/drive/MyDrive/thesis/dataset/cleaned_df/df_cnn.csv')
df_cnn.head()

,ds,y,temp,rad,preci
0,2007-01-01 00:00:00,0.226,10.00,0.0,2.6
1,2007-01-01 01:00:00,0.248,10.58,0.0,0.8
2,2007-01-01 02:00:00,0.248,11.22,0.0,0.2
3,2007-01-01 03:00:00,0.320,11.42,0.0,0.6
4,2007-01-01 04:00:00,0.346,11.58,0.0,0.0


## Data Processing

In [17]:
# Data Processing for Multivariable CNN - Small Sample Size for Testing
df_cnn = df_cnn.set_index('ds')
df_cnn = df_cnn[['temp', 'rad', 'preci', 'y']]
df_cnn = df_cnn.to_numpy()

In [18]:
# CNN - split into train and test sets
n = 9*365*24
trs_cnn = df_cnn[:n, :]
tes_cnn = df_cnn[n:, :]

In [19]:
# CNN Data Processing - split multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# choose a number of time steps
n_steps = 3
# convert training set into input/output
trs_cnn_X, trs_cnn_y = split_sequences(trs_cnn, n_steps)
print(trs_cnn_X.shape, trs_cnn_y.shape)
# convert testing set into input/output
tes_cnn_X, tes_cnn_y = split_sequences(tes_cnn, n_steps)
print(tes_cnn_X.shape, tes_cnn_y.shape)

# define number of feature variables
n_features = trs_cnn_X.shape[2]

(78838, 3, 3) (78838,)
(26311, 3, 3) (26311,)


## Prediction

In [20]:
# CNN Model Definition
model_cnn = Sequential()
model_cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps, n_features)))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(50, activation='relu'))
model_cnn.add(Dense(1))
model_cnn.compile(optimizer='adam', loss='mse')

## Evaluation

In [21]:
# CNN Fit Model
cnn_history = model_cnn.fit(trs_cnn_X, trs_cnn_y, validation_data=(tes_cnn_X, tes_cnn_y), epochs=200, verbose=2)


Epoch 1/200
2464/2464 - 5s - loss: 0.8676 - val_loss: 1.5084 - 5s/epoch - 2ms/step
Epoch 2/200
2464/2464 - 5s - loss: 0.2483 - val_loss: 0.0802 - 5s/epoch - 2ms/step
Epoch 3/200
2464/2464 - 5s - loss: 0.1044 - val_loss: 0.0700 - 5s/epoch - 2ms/step
Epoch 4/200
2464/2464 - 4s - loss: 0.0895 - val_loss: 0.0710 - 4s/epoch - 2ms/step
Epoch 5/200
2464/2464 - 4s - loss: 0.0752 - val_loss: 0.0716 - 4s/epoch - 2ms/step
Epoch 6/200
2464/2464 - 4s - loss: 0.0604 - val_loss: 0.0615 - 4s/epoch - 2ms/step
Epoch 7/200
2464/2464 - 4s - loss: 0.0569 - val_loss: 0.0633 - 4s/epoch - 2ms/step
Epoch 8/200
2464/2464 - 5s - loss: 0.0548 - val_loss: 0.0632 - 5s/epoch - 2ms/step
Epoch 9/200
2464/2464 - 5s - loss: 0.0524 - val_loss: 0.0612 - 5s/epoch - 2ms/step
Epoch 10/200
2464/2464 - 5s - loss: 0.0518 - val_loss: 0.0633 - 5s/epoch - 2ms/step
Epoch 11/200
2464/2464 - 4s - loss: 0.0509 - val_loss: 0.0646 - 4s/epoch - 2ms/step
Epoch 12/200
2464/2464 - 5s - loss: 0.0507 - val_loss: 0.0599 - 5s/epoch - 2ms/step
E

In [22]:
# CNN Prediction
trs_cnn_pred = model_cnn.predict(trs_cnn_X)
tes_cnn_pred = model_cnn.predict(tes_cnn_X)
# print('Train rmse:', np.sqrt(mean_squared_error(trs_cnn_y, trs_cnn_pred)))
# print('Validation rmse:', np.sqrt(mean_squared_error(tes_cnn_y, tes_cnn_pred)))


In [23]:
trs_cnn_pred = trs_cnn_pred.reshape(-1,1)
trs_cnn_y = trs_cnn_y.reshape(-1,1)
tes_cnn_pred = tes_cnn_pred.reshape(-1,1)
tes_cnn_y = tes_cnn_y.reshape(-1,1)

In [24]:
# calculate NSE
nse_trs_cnn = 1-(np.sum((trs_cnn_pred-trs_cnn_y)**2)/np.sum((trs_cnn_y-np.mean(trs_cnn_y))**2))
nse_tes_cnn = 1-(np.sum((tes_cnn_pred-tes_cnn_y)**2)/np.sum((tes_cnn_y-np.mean(tes_cnn_y))**2))
print('Train NSE: %.3f' % nse_trs_cnn)
print('Test NSE: %.3f' % nse_tes_cnn)

# calculate RMSE
rmse_trs_cnn = np.sqrt(mean_squared_error(trs_cnn_y, trs_cnn_pred))
rmse_tes_cnn = np.sqrt(mean_squared_error(tes_cnn_y, tes_cnn_pred))
print('Train RMSE: %.3f' % rmse_trs_cnn)
print('Test RMSE: %.3f' % rmse_tes_cnn)

Train NSE: 0.209
Test NSE: 0.189
Train RMSE: 0.215
Test RMSE: 0.244


# Archive

In [25]:
# # NSE Calculation and Plot

# o = np.array([1,2,3,4,5,6,7,8,9,10,2,3,4,5,6,7])
# m = np.array([1.1,2.2,3.2,4,5,6.1,7.2,8.5,8,10.5,1,2,4,5,6,7])
# # nse = 1-(np.sum((p-t)**2)/np.sum((t-np.mean(t))**2))
# # print('Test NSE: %.3f' % nse)
# # plot


# fig= plt.figure(figsize=(14, 4))
# plt.title('Insert Title')
# plt.plot(o, label='observed', color='#00688b', linewidth=0.5)
# plt.plot(m, label='model', color='#ee7600', linewidth=0.5)
# plt.plot([], [], ' ', label='NSE = %.3f' % nse)
# plt.ylabel('y label')
# plt.ylabel('x label')
# plt.legend()
# plt.show()



In [26]:
# # Data Processing for Multivariable CNN - Small Sample Size for Testing
# df_cnn = df_cnn.iloc[:2500, :]
# df_cnn = df_cnn.set_index('ds')
# df_cnn = df_cnn[['temp', 'rad', 'preci', 'y']]
# df_cnn = df_cnn.to_numpy()